In [ ]:
# making all the needed imports
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.layers import LSTM, Dense
from google.colab import files
import pickle


In [ ]:
# downloading stock data
data = yf.download('AAPL', start='1990-01-01', end='2024-01-01')
data = data[['Close']]
print("done downloading data")

# scaling data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)
print("done scaling")

# preprocessing data
X, y = [], []
for i in range(60, len(scaled_data)):
    X.append(scaled_data[i-60:i, 0])
    y.append(scaled_data[i, 0])
X, y = np.array(X), np.array(y)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))
print("done preprocessing")

# building & training model
# I decided to use LSTM since it works well with Time Series Analysis
model = Sequential()

# Adding LSTM layers with specified initializers so I can set it to known values
model.add(LSTM(units= 50, return_sequences= True, input_shape= (X.shape[1], 1), kernel_initializer= GlorotUniform()))
model.add(LSTM(units= 50, kernel_initializer= GlorotUniform()))
model.add(Dense(1, kernel_initializer= GlorotUniform()))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X, y, epochs=10, batch_size=32)
print("done training & fitting")

# saving model & scaler
model.save('trading_model.h5')
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

files.download('trading_model.h5')
files.download('scaler.pkl')
print("done saving files")